# 5. Segregation Indexes

## Prerequistes

In [3]:
import numpy as np
import pandas as pd
import pickle
import graph_tool as gt
import scipy.sparse as sp

In [5]:
grafo = gt.load_graph('/mnt/disk2/Data/Daily_graphs/2021-04-28.graphml')
adjacency = gt.spectral.adjacency(grafo)
with open('/mnt/disk2/Data/Pickle/types_matrix.pkl') as file:
    types_matrix = pickle.load(file)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte